<a href="https://colab.research.google.com/github/shwoa/Capstone/blob/main/Tabnet_%EA%B2%B0%EC%B8%A1%EC%B9%98%EC%B5%9C%EB%B9%88%EB%8C%80%EC%B2%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 토마토_표현형.CSV to 토마토_표현형.CSV


# **MIR**



1.   토마토_표현형.CSV
2.   유전형인코딩_최빈값.csv
3.   MutualInfo_SNP()



In [ ]:
!pip install pytorch-tabnet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from pytorch_tabnet.tab_model import TabNetRegressor
from IPython.display import display

# ✅ 경로 설정
geno_path = "유전형인코딩_최빈값.csv"
pheno_path = "토마토_표현형.CSV"
mi_files = {
    "Top 100": "MutualInfo_SNP(100).csv",
    "Top 500": "MutualInfo_SNP(500).csv",
    "Top 1000": "MutualInfo_SNP(1000).csv",
    "Top 3000": "MutualInfo_SNP(3000).csv",
    "Top 5000": "MutualInfo_SNP(5000).csv",
    "Top 10000": "MutualInfo_SNP(10000).csv"
}

# ✅ 유전형 데이터 불러오기 (전치 필수)
geno_df = pd.read_csv(geno_path, index_col=0).T
geno_df.index = geno_df.index.astype(str).str.strip()
geno_df.columns = geno_df.columns.astype(str).str.strip()

# ✅ 표현형 데이터
pheno_df = pd.read_csv(pheno_path, index_col=0, encoding='cp949')  # 필요시 encoding 수정
pheno_df.columns = pheno_df.columns.str.strip()
pheno_df.index = pheno_df.index.astype(str).str.strip()
phenotypes = pheno_df.columns.tolist()

# ✅ 결과 저장
results = []

# ✅ TabNet 학습 반복
for pheno in phenotypes:
    y_all = pheno_df[pheno].dropna()

    for label, file in mi_files.items():
        try:
            mi_df = pd.read_csv(file)
            mi_df.columns = mi_df.columns.str.strip()
            mi_df["SNP"] = mi_df["SNP"].astype(str).str.strip()

            # SNP 리스트 추출
            snps = [s for s in mi_df["SNP"] if s in geno_df.columns]
            if len(snps) == 0:
                raise ValueError("사용 가능한 SNP 없음")

            # X, y 병합
            X = geno_df[snps].loc[y_all.index].dropna()
            y = y_all.loc[X.index]

            # 정규화
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)

            # 분할
            X_train, X_test, y_train, y_test = train_test_split(
                X_scaled, y, test_size=0.2, random_state=42
            )

            # TabNet 학습
            model = TabNetRegressor(verbose=0)
            model.fit(
                X_train=X_train, y_train=y_train.values.reshape(-1, 1),
                eval_set=[(X_test, y_test.values.reshape(-1, 1))],
                eval_metric=['rmse'],
                max_epochs=100,
                patience=10,
                batch_size=256,
                virtual_batch_size=128,
                num_workers=0,
                drop_last=False
            )

            # 평가
            y_pred = model.predict(X_test).flatten()
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            r2 = r2_score(y_test, y_pred)

            results.append({
                "표현형": pheno,
                "SNP 개수": label,
                "R²": round(r2, 4),
                "RMSE": round(rmse, 4)
            })

        except Exception as e:
            print(f"❌ {pheno} - {label} 에러: {e}")
            results.append({
                "표현형": pheno,
                "SNP 개수": label,
                "R²": "에러",
                "RMSE": "에러"
            })

# # ✅ 결과 출력
# results_df = pd.DataFrame(results)
# results_pivot = results_df.pivot(index="표현형", columns="SNP 개수", values="R²")
# print("📊 TabNet 성능 비교표 (R² 기준):")
# display(results_pivot)

# ✅ 결과 출력
results_df = pd.DataFrame(results)

# 원하는 순서로 열 정렬
desired_order = ["Top 100", "Top 500", "Top 1000", "Top 3000", "Top 5000", "Top 10000"]
results_pivot = results_df.pivot(index="표현형", columns="SNP 개수", values="R²")
results_pivot = results_pivot[desired_order]  # 열 순서 지정

print("📊 TabNet 성능 비교표 (R² 기준):")
display(results_pivot)




Early stopping occurred at epoch 80 with best_epoch = 70 and best_val_0_rmse = 43.3262


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 52 with best_epoch = 42 and best_val_0_rmse = 56.03487


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 62 with best_epoch = 52 and best_val_0_rmse = 45.12891


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 58 with best_epoch = 48 and best_val_0_rmse = 52.53349


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 67 with best_epoch = 57 and best_val_0_rmse = 45.33057


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


# **GWAS**



1.   토마토_표현형.CSV
2.   유전형인코딩_최빈값.csv
3.   GWAS_SNP()



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from pytorch_tabnet.tab_model import TabNetRegressor
from IPython.display import display

# ✅ 경로 설정
geno_path = "유전형인코딩_최빈값.csv"
pheno_path = "토마토_표현형.CSV"
gwas_files = {
    "Top 20": "GWAS_SNP(20).csv",
    "Top 50": "GWAS_SNP(50).csv",
    "Top 100": "GWAS_SNP(100).csv",
    "Top 500": "GWAS_SNP(500).csv",
    "Top 1000": "GWAS_SNP(1000).csv",
    "Top 2000": "GWAS_SNP(2000).csv"
}

# ✅ 1. 표현형 이름 매핑
trait_map = {
    "과중": "과중 (g)",
    "과장": "과장 (mm)",
    "과폭": "과폭 (mm)",
    "과피두께": "과피두께 (mm)",
    "당도": "당도 (%)",
    "과실경도": "과실경도 (kg)"
}

# ✅ 2. 유전형 데이터 로딩
geno_df = pd.read_csv(geno_path, index_col=0).T
geno_df.index = geno_df.index.astype(str).str.strip()
geno_df.columns = geno_df.columns.astype(str).str.strip()

# ✅ 3. 표현형 데이터 로딩
pheno_df = pd.read_csv(pheno_path, index_col=0, encoding='cp949')
pheno_df.columns = pheno_df.columns.str.strip()
pheno_df.index = pheno_df.index.astype(str).str.strip()

# ✅ 4. 결과 저장 리스트
results = []

# ✅ 5. 표현형 반복
for trait, pheno_col in trait_map.items():
    if pheno_col not in pheno_df.columns:
        print(f"⚠️ 표현형 열 누락: {pheno_col}")
        continue

    y_all = pheno_df[pheno_col].dropna()

    for label, file in gwas_files.items():
        try:
            gwas_df = pd.read_csv(file)
            gwas_df.columns = gwas_df.columns.str.strip()
            gwas_df["SNP"] = gwas_df["SNP"].astype(str).str.strip()

            # GWAS는 trait별 구분 없음 → SNP 전체 사용
            snps = [s for s in gwas_df["SNP"] if s in geno_df.columns]
            if len(snps) == 0:
                raise ValueError("사용 가능한 SNP 없음")

            # X, y 병합
            X = geno_df[snps].loc[y_all.index].dropna()
            y = y_all.loc[X.index]

            # 정규화
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X).astype(np.float32)

            # 학습용 분할
            X_train, X_test, y_train, y_test = train_test_split(
                X_scaled, y, test_size=0.2, random_state=42
            )

            # TabNet 학습
            model = TabNetRegressor(verbose=0)
            model.fit(
                X_train=X_train, y_train=y_train.values.reshape(-1, 1),
                eval_set=[(X_test, y_test.values.reshape(-1, 1))],
                eval_metric=['rmse'],
                max_epochs=100,
                patience=10,
                batch_size=64,
                virtual_batch_size=32,
                num_workers=0,
                drop_last=False
            )

            # 예측 및 평가
            y_pred = model.predict(X_test).flatten()
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            r2 = r2_score(y_test, y_pred)

            results.append({
                "표현형": trait,
                "SNP 개수": label,
                "R²": round(r2, 4),
                "RMSE": round(rmse, 4)
            })

        except Exception as e:
            print(f"❌ {trait} - {label} 에러: {e}")
            results.append({
                "표현형": trait,
                "SNP 개수": label,
                "R²": "에러",
                "RMSE": "에러"
            })

# ✅ 6. 결과 출력
results_df = pd.DataFrame(results)
results_r2 = results_df.pivot(index="표현형", columns="SNP 개수", values="R²")
results_rmse = results_df.pivot(index="표현형", columns="SNP 개수", values="RMSE")

print("📊 TabNet 성능 비교표 (R² 기준):")
display(results_r2)

print("📉 TabNet 성능 비교표 (RMSE 기준):")
display(results_rmse)



Early stopping occurred at epoch 28 with best_epoch = 18 and best_val_0_rmse = 48.43376


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 26 with best_epoch = 16 and best_val_0_rmse = 50.37766


/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 33 with best_epoch = 23 and best_val_0_rmse = 49.74193


...

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


📊 TabNet 성능 비교표 (R² 기준):


SNP 개수,Top 100,Top 1000,Top 20,Top 2000,Top 50,Top 500
표현형,,,,,,
과실경도,0.0323,-2.2948,0.1445,0.1410,0.0257,0.1824
과장,0.3980,0.1270,0.4662,0.5785,0.4624,-0.1859
과중,0.3063,0.4146,0.3423,0.3378,0.2885,0.4725
과폭,0.4495,0.2753,0.5513,0.4029,0.5316,0.4223
과피두께,0.5441,0.2207,-0.7781,0.2058,0.3356,-1.5609
당도,0.4228,0.4044,0.2608,0.4600,0.3715,0.3374


📉 TabNet 성능 비교표 (RMSE 기준):


SNP 개수,Top 100,Top 1000,Top 20,Top 2000,Top 50,Top 500
표현형,,,,,,
과실경도,0.1112,0.2051,0.1045,0.1047,0.1116,0.1022
과장,8.5558,10.3027,8.0562,7.1593,8.0849,12.0081
과중,49.7419,45.6960,48.4338,48.5999,50.3777,43.3754
과폭,11.7234,13.4520,10.5846,12.2101,10.8143,12.0103
과피두께,0.6607,0.8639,1.3049,0.8721,0.7977,1.5661
당도,0.9025,0.9168,1.0213,0.8730,0.9418,0.9670
